In [121]:
import pandas as pd
import numpy
import requests
# import json
import re
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from time import sleep

In [25]:
HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

#Using Selenium

options = Options()
options.add_argument("--headless")

wd_path = "geckodriver.exe"
wdriver = webdriver.Firefox(options=options)
wdriver.get("https://www.youtube.com/feed/trending")
sleep(3)

dynamically_loaded_page = wdriver.page_source
print(dynamically_loaded_page)


wdriver.quit()

<html style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" lang="en" darker-dark-theme="" darker-dark-theme-deprecate="" dark="" system-icons="" typography="" typography-spacing=""><head><script data-original-src="/s/player/da154528/player_ias.vflset/en_US/miniplayer.js" nonce="GXxkMtQ3epMcvUzoauVDBw" src="/s/player/da154528/player_ias.vflset/en_US/miniplayer.js"></script><script data-original-src="/s/player/da154528/player_ias.vflset/en_US/remote.js" nonce="GXxkMtQ3epMcvUzoauVDBw" src="/s/player/da154528/player_ias.vflset/en_US/remote.js"></script><script data-original-src="/s/player/da154528/player_ias.vflset/en_US/offline.js" nonce="GXxkMtQ3epMcvUzoauVDBw" src="/s/player/da154528/player_ias.vflset/en_US/offline.js"></script><script data-id="_gd" nonce="GXxkMtQ3epMcvUzoauVDBw">window.WIZ_global_data = {"MuJWjd":false,"nQyAE":{},"oxN3nb":{"1":false}};</script><meta http-equiv="origin-trial" content="ApvK67ociHgr2egd6c2ZjrfPuRs8BHcvSggogIOPQNH7GJ3cVlyJ1NOq/COCdj0+zxskqHt9HgL

In [133]:
soup = BeautifulSoup(dynamically_loaded_page, 'html.parser')

data = []
links = []
titles = []
videoTitles = soup.find_all("a", id="video-title")
for video in videoTitles:
    data.append(video["aria-label"])
    links.append("https://www.youtube.com"+video["href"])
    titles.append(video["title"])

data = [datum.replace(f"{titles[i]} by ", "") for i, datum in enumerate(data)]

numberOfViewers = []
uploadedTimes = []
channelNames = []
for datum in data:
    channel, viewers, uploadedTime = re.findall(r"^(.*)\s([0-9,]+)\sviews\s([0-9a-zA-Z\s,]+)$", datum)[0]
    channelNames.append(channel)
    numberOfViewers.append(viewers)
    uploadedTimes.append(uploadedTime)
    
#Parsing View Counts and Upload Time
numberOfViewers= [int(re.sub(r",", "", view)) for view in numberOfViewers]
# print(uploadedTimes)

uploadedDates = []

for time in uploadedTimes:
    #Not the most accurate year and months count, but close enough to work for the purpose of analysis
    years = re.findall(r"([0-9]+)\syear", time)
    years = int(years[0]) if years else 0
    months = re.findall(r"([0-9]+)\smonth", time)
    months = int(months[0]) if months else 0
    weeks = re.findall(r"([0-9]+)\sweek", time)
    weeks = int(weeks[0]) if weeks else 0
    days = re.findall(r"([0-9]+)\sday", time)
    days = (int(days[0])+(years*365)+(months*30)) if days else (years*365)+(months*30)
    hours = re.findall(r"([0-9]+)\shour", time)
    hours = int(hours[0]) if hours else 0
    minutes = re.findall(r"([0-9]+)\sminute", time)
    minutes = int(minutes[0]) if minutes else 0
    seconds = re.findall(r"([0-9]+)\ssecond", time)
    seconds = int(seconds[0]) if seconds else 0
    
    uploadedDates.append(datetime.today()-timedelta(weeks=weeks, days=days, hours=hours, minutes=minutes, seconds=seconds))


print(len(links))    
print(len(titles))
print(len(channelNames))
print(len(numberOfViewers))
print(len(uploadedDates))

97
97
97
97
97


In [132]:
dic = {"title": titles, "channelName": channelNames, "viewCount": numberOfViewers, "uploadDate": uploadedDates, "link": links}
df = pd.DataFrame(data=dic)
df = df.astype({'title':'string', 'channelName':'string', 'viewCount':'int64', 'link':'string', 'uploadDate': 'datetime64[ns]'})
df.describe()

,viewCount,uploadDate
count,9.700000e+01,97
mean,1.942491e+06,2023-12-19 12:24:57.495019776
min,9.923700e+04,2023-12-08 18:30:13.010968
25%,3.294340e+05,2023-12-18 18:07:50.010968064
50%,5.827450e+05,2023-12-20 18:20:50.009968128
75%,1.434066e+06,2023-12-21 18:17:50.009968128
max,9.081174e+07,2023-12-22 00:57:50.009968
std,9.177968e+06,NaN
